In [ ]:
### non cancellare...importa su tutti i file

if torch.cuda.is_available():
    print('Cuda Available')
else:
    print('No GPU support currently')

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import pandas as pd
import numpy as np
import torch
import copy

# Caricamento e filtraggio dei dati
file_name = ('DB_FINALE_CORRETTO_PER_DEEPL_con_proxy_morfologici_RSLOPEaspect.csv')
data = pd.read_csv(file_name)

# Rimuovi le righe con valori NaN
data_cleaned = data.dropna()
data_filtered = data_cleaned[data_cleaned['spessore'] > 29]

# Creazione dei nuovi database con mutazioni
data_base = data_filtered.copy()
data_base['Database'] = 'Base'

data_quota_mutata = data_filtered.copy()
data_quota_mutata['DEM'] = 0
data_quota_mutata['Database'] = 'QuotaMutata'

data_slope_mutato = data_filtered.copy()
data_slope_mutato['Slope'] = 0
data_slope_mutato['Database'] = 'SlopeMutato'

data_roughness_mutato = data_filtered.copy()
data_roughness_mutato['Roughness'] = 0
data_roughness_mutato['Database'] = 'RoughnessMutato'

data_ruggedness_mutato = data_filtered.copy()
data_ruggedness_mutato['Ruggedness'] = 0
data_ruggedness_mutato['Database'] = 'RuggednessMutato'

data_Topographic_position_index1_mutato = data_filtered.copy()
data_Topographic_position_index1_mutato['Topographic_position_index'] = 0
data_Topographic_position_index1_mutato['Database'] = 'Topographic_position_index1_Mutato'

data_Profile_curvature_mutato = data_filtered.copy()
data_Profile_curvature_mutato['Profile_curvature'] = 0
data_Profile_curvature_mutato['Database'] = 'Profile_curvature'

data_Tangential_curvature_mutato = data_filtered.copy()
data_Tangential_curvature_mutato['Tangential_curvature'] = 0
data_Tangential_curvature_mutato['Database'] = 'Tangential_curvature'

data_Aspect_mutato = data_filtered.copy()
data_Aspect_mutato['Aspect'] = 0
data_Aspect_mutato['Database'] = 'Aspect'

data_Dx_mutato = data_filtered.copy()
data_Dx_mutato['Dx'] = 0
data_Dx_mutato['Database'] = 'Dx'

data_Dy_mutato = data_filtered.copy()
data_Dy_mutato['Dy'] = 0
data_Dy_mutato['Database'] = 'Dy'

# Concatenazione dei database, NB ricorda che questo db è x5 non x3
data_triplicato = pd.concat([data_base, data_quota_mutata, data_slope_mutato,data_roughness_mutato,data_ruggedness_mutato,data_Topographic_position_index1_mutato, 
                             data_Profile_curvature_mutato, data_Tangential_curvature_mutato, data_Aspect_mutato, data_Dx_mutato, data_Dy_mutato], ignore_index=True)bb

In [ ]:
#### versione 1.2 con 2 strati hidden

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

# Estrai le colonne necessarie (latitudine, longitudine, quota, tipo di roccia, slope, Vs30)
features = data_triplicato[['Acoord_X', 'Acoord_Y', 'DEM', 'Geologia', 'Slope', 'Roughness', 'Ruggedness', 'Topographic_position_index',
                           'Profile_curvature', 'Tangential_curvature', 'Aspect', 'Dx', 'Dy']]
labels = data_triplicato[['Vs']]

# Normalizzazione dei dati
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features.values) 

# Divisione dei dati in set di addestramento, validazione e test
features_train, features_temp, labels_train, labels_temp = train_test_split(
    features_normalized, labels.values, test_size=0.2, random_state=42)

features_val, features_test, labels_val, labels_test = train_test_split(
    features_temp, labels_temp, test_size=0.5, random_state=42)

# Converti i dati in tensori PyTorch
features_tensor_train = torch.tensor(features_train, dtype=torch.float32)
labels_tensor_train = torch.tensor(labels_train, dtype=torch.float32)
features_tensor_val = torch.tensor(features_val, dtype=torch.float32)
labels_tensor_val = torch.tensor(labels_val, dtype=torch.float32)
features_tensor_test = torch.tensor(features_test, dtype=torch.float32)
labels_tensor_test = torch.tensor(labels_test, dtype=torch.float32)

# Creazione del DataLoader per gestire i dati di addestramento
train_dataset = TensorDataset(features_tensor_train, labels_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Creazione di un DataLoader per i dati di validazione
val_dataset = TensorDataset(features_tensor_val, labels_tensor_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Creazione di un DataLoader per i dati di test
test_dataset = TensorDataset(features_tensor_test, labels_tensor_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Definisci la rete neurale con dropout
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_prob=0.01):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        self.layer3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.layer3(x)
        return x

# Parametri della rete
l1_lambda = 0.00001
l2_lambda = 0.00001
input_size = 13  # Numero di features di input
output_size = 1  # Output è la stima della Vs30

# Tuning degli iperparametri
hidden_sizes = [(4096, 8192)]
learning_rates = [0.01]
best_model = None
best_loss = float('inf')

# Colori diversi per le curve
colors = ['red', 'blue', 'green', 'purple', 'orange']

diagnostic = []

# Aggiungi la regolarizzazione L1 e L2 al calcolo della perdita
def loss_with_regularization(outputs, labels, l1_lambda, l2_lambda):
    l1_reg = 0
    l2_reg = 0
    for param in model.parameters():
        l1_reg += torch.norm(param, 1)
        l2_reg += torch.norm(param, 2)
    loss = criterion(outputs, labels)
    loss += l1_lambda * l1_reg
    loss += l2_lambda * l2_reg
    return loss

plt.figure(figsize=(10, 6))
for i, (hidden_size1, hidden_size2, ) in enumerate(hidden_sizes):
  print(hidden_size1,hidden_size2,)
  for j, lr in enumerate(learning_rates):
        color_index = i * len(learning_rates) + j
        if color_index < len(colors):
            color = colors[color_index]
        else:
            color = 'black'  # Usa il nero se non ci sono abbastanza colori definiti
        ### da quì metti tutto in a function
        # Creazione della rete con i parametri corretti
        model = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size).to(device)

        # Definizione della funzione di perdita e dell'ottimizzatore
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=l2_lambda)

        # Definisci il learning rate scheduler
        scheduler = StepLR(optimizer, step_size=100, gamma=0.1)  # Diminuisci il tasso di apprendimento ogni 100 epoche

        # Liste separate per la configurazione corrente
        train_losses_config = []
        val_losses_config = []

        # Addestramento della rete con regolarizzazione e scheduler
        num_epochs = 500 # 50 era la prima....poi 200
        for epoch in range(num_epochs):
            model.train()  
            train_loss_epoch = 0.0  
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                inputs, labels = inputs.to(device), labels.to(device)  # Move tensors to the configured device
                outputs = model(inputs)
                loss = loss_with_regularization(outputs, labels, l1_lambda, l2_lambda)  # Pass outputs and labels to the function
                loss.backward()
                optimizer.step()
                train_loss_epoch += loss.item()
            scheduler.step()  # Applica il learning rate scheduler
            model.eval()

            # Calcola la perdita media sul set di addestramento per questa epoca
            train_loss_epoch /= len(train_loader)
            train_losses_config.append(train_loss_epoch)

            # Calcola la perdita sul set di validazione
            val_loss_epoch = 0.0  # Resetta la perdita ad ogni epoca
            num_batches_val = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs,labels = inputs.to(device),labels.to(device)
                    outputs = model(inputs)
                    val_loss_epoch += criterion(outputs, labels).item()
                    num_batches_val += 1

            # Calcola la perdita media sul set di validazione per questa epoca
            val_loss_epoch /= num_batches_val
            val_losses_config.append(val_loss_epoch)

            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss_epoch:.4f}')

            # Valuta le prestazioni del modello sul set di validazione
            if val_loss_epoch < best_loss:
                best_loss = val_loss_epoch
                best_epoch = epoch
                best_model = copy.deepcopy(model)
            # return best loss, best epoch, model? 
        diagnostic.append([i, lr, best_loss, best_epoch])
        # Visualizza la perdita su train e validation per ogni configurazione
        plt.plot(range(1, num_epochs + 1), train_losses_config, label=f'Train Loss (H1={hidden_size1}, H2={hidden_size2}, LR={lr})', color=color)
        plt.plot(range(1, num_epochs + 1), val_losses_config, linestyle='--', label=f'Validation Loss (H1={hidden_size1}, H2={hidden_size2}, LR={lr})', color=color)

        # Aggiungi un marcatore per il punto di minima perdita sul set di validazione
        min_val_loss_epoch = np.argmin(val_losses_config) + 1
        min_val_loss = min(val_losses_config)
        plt.scatter(min_val_loss_epoch, min_val_loss, color=color, marker='o', s=100,
                    label=f'Min Val Loss (H1={hidden_size1}, H2={hidden_size2}, LR={lr})')

# Visualizza i grafici di perdita
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Posiziona la legenda fuori dal grafico
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            predictions.extend(outputs.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    average_loss = total_loss / len(dataloader)
    rmse = np.sqrt(mean_squared_error(true_labels, predictions))
    r2 = r2_score(true_labels, predictions)

    return average_loss, rmse, r2, predictions, true_labels

# Valuta il modello sui set di addestramento, validazione e test
train_loss, train_rmse, train_r2, train_predictions, train_true_labels = evaluate_model(best_model, train_loader, nn.MSELoss(), device)
val_loss, val_rmse, val_r2, val_predictions, val_true_labels = evaluate_model(best_model, val_loader, nn.MSELoss(), device)
test_loss, test_rmse, test_r2, test_predictions, test_true_labels = evaluate_model(best_model, test_loader, nn.MSELoss(), device)

print(f'Train Loss: {train_loss:.4f}, Train RMSE: {train_rmse:.4f}, Train R^2 Score: {train_r2:.4f}')
print(f'Validation Loss: {val_loss:.4f}, Validation RMSE: {val_rmse:.4f}, Validation R^2 Score: {val_r2:.4f}')
print(f'Test Loss: {test_loss:.4f}, Test RMSE: {test_rmse:.4f}, Test R^2 Score: {test_r2:.4f}')

# Plotta i valori previsti rispetto ai valori reali per il set di test
plt.figure(figsize=(8, 8))
plt.scatter(test_true_labels, test_predictions, color='blue', alpha=0.7, edgecolor='black')
plt.plot([min(test_true_labels), max(test_true_labels)], [min(test_true_labels), max(test_true_labels)], linestyle='--', color='red', linewidth=2)
plt.xlabel('True value')
plt.ylabel('Predicted value')
plt.title('Scatter plot Real vs. Predicted (Test Set)')
plt.grid(True)
plt.show()